In [134]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast 

In [135]:
df = pd.read_csv(r'C:\Users\zinhyeok\Developer\gbottomup\results\multi_point_GraphStep\20250821_145247_5\muliple_test_model16_d500_frequency_50_20250821_145247.csv')

In [136]:
df.head()

,scenario,seed,model,G_bottomup_tree,True_tau,G_multi_tau,G_bottomup_tau,G_multi_n_cps,G_bottomup_n_cps,G_multi_ari,G_bottomup_ari
0,model16,0,premerge_True_c_2_None_refineFalse_minobs4_mod...,"{1: [(1, np.float64(0.0)), (3, np.float64(0.0)...","[49, 99, 109, 149]","[49, 99, 109, 149]","[7, 25, 47, 73, 79, 83, 97, 99, 109, 143, 149]",4,11,1.000000,0.690384
1,model16,0,premerge_True_c_2_None_refineFalse_minobs6_mod...,"{1: [(2, np.float64(0.0)), (5, np.float64(0.0)...","[49, 99, 109, 149]","[49, 99, 109, 149]","[38, 50, 110, 125, 131, 149, 152, 167, 182, 185]",4,10,1.000000,0.599791
2,model16,0,premerge_True_c_2_None_refineFalse_minobs8_mod...,"{1: [(3, np.float64(0.0)), (7, np.float64(3.01...","[49, 99, 109, 149]","[49, 99, 109, 149]","[19, 47, 79, 83, 99, 107, 151, 183, 191]",4,9,1.000000,0.624897
3,model16,0,premerge_True_c_2_None_refineFalse_minobs10_mo...,"{1: [(4, np.float64(0.0)), (9, np.float64(1.05...","[49, 99, 109, 149]","[49, 99, 109, 149]","[29, 34, 49, 99, 104, 109, 124, 149, 154]",4,9,1.000000,0.794801
4,model16,1,premerge_True_c_2_None_refineFalse_minobs4_mod...,"{1: [(1, np.float64(0.0)), (3, np.float64(0.0)...","[49, 99, 109, 149]","[46, 49, 99, 102, 109, 149]","[15, 31, 49, 79, 101, 109, 119, 135, 137, 151,...",6,12,0.976886,0.472413


In [137]:
unique_models = df['model'].unique()
unique_models

array(['premerge_True_c_2_None_refineFalse_minobs4_modeltypebase_numcp5',
       'premerge_True_c_2_None_refineFalse_minobs6_modeltypebase_numcp5',
       'premerge_True_c_2_None_refineFalse_minobs8_modeltypebase_numcp5',
       'premerge_True_c_2_None_refineFalse_minobs10_modeltypebase_numcp5'],
      dtype=object)

In [138]:
# --- 0. 모델 이름 매핑 ---
model_name_mapping = {
    'premerge_True_c_2_None_refineFalse_minobs4_modeltypebase_numcp5': 'Model A (minobs=4)',
    'premerge_True_c_2_None_refineFalse_minobs6_modeltypebase_numcp5': 'Model B (minobs=6)',
    'premerge_True_c_2_None_refineFalse_minobs8_modeltypebase_numcp5': 'Model C (minobs=8)',
    'premerge_True_c_2_None_refineFalse_minobs10_modeltypebase_numcp5': 'Model D (minobs=10)',
}
df['model_name'] = df['model'].map(model_name_mapping).fillna(df['model'])

In [139]:
def parse_tau_list(series):
    new_series = []
    for item in series:
        try:
            parsed_list = ast.literal_eval(item)
            if isinstance(parsed_list, list):
                if parsed_list and isinstance(parsed_list[0], tuple):
                    new_series.append([int(t[0]) for t in parsed_list])
                else:
                    new_series.append([int(t) for t in parsed_list])
            else:
                 new_series.append([])
        except (ValueError, SyntaxError):
            new_series.append([])
    return new_series

df['True_tau_list'] = parse_tau_list(df['True_tau'])
df['G_multi_tau_list'] = parse_tau_list(df['G_multi_tau'])
df['G_bottomup_tau_list'] = parse_tau_list(df['G_bottomup_tau'])

In [140]:
df_sbs_candidates = df[['seed', 'G_multi_tau_list']].drop_duplicates(subset=['seed'])
df_sbs_candidates.rename(columns={'G_multi_tau_list': 'sbs_candidate_list'}, inplace=True)

# 기존 데이터에 SBS 후보군을 seed 기준으로 합침
df = pd.merge(df, df_sbs_candidates, on='seed')

In [141]:
# --- 평가 지표 계산 함수 ---
TOLERANCE = 5

In [142]:
def calculate_set_metrics(true_taus, candidate_taus, tol):
    if not true_taus and not candidate_taus: return 1.0, 1.0, 1.0
    if not true_taus: return 1.0, 0.0, 0.0
    if not candidate_taus: return 0.0, 1.0, 0.0
    tp = min(len(true_taus), sum(1 for tt in true_taus if any(abs(tt - ct) <= tol for ct in candidate_taus)))
    fp = sum(1 for ct in candidate_taus if not any(abs(ct - tt) <= tol for tt in true_taus))
    fn = len(true_taus) - tp
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    return recall, precision, f1

In [143]:
def calculate_modified_jaccard(set_a, set_b, tol):
    if not set_a and not set_b: return 1.0
    matches_a = {i for i, a in enumerate(set_a) if any(abs(a - b) <= tol for b in set_b)}
    matches_b = {i for i, b in enumerate(set_b) if any(abs(b - a) <= tol for a in set_a)}
    intersection_size = (len(matches_a) + len(matches_b)) / 2.0
    union_size = len(set_a) + len(set_b) - intersection_size
    return intersection_size / union_size if union_size > 0 else 0.0

In [144]:
df_main_models = df[['seed', 'model_name', 'True_tau_list', 'G_bottomup_tau_list']]
df_main_models.rename(columns={'G_bottomup_tau_list': 'candidate_tau_list'}, inplace=True)
df_sbs = df[['seed', 'True_tau_list', 'sbs_candidate_list']].drop_duplicates(subset=['seed'])
df_sbs['model_name'] = 'SBS_Baseline'
df_sbs.rename(columns={'sbs_candidate_list': 'candidate_tau_list'}, inplace=True)
df_all = pd.concat([df_main_models, df_sbs], ignore_index=True)

C:\Users\zinhyeok\AppData\Local\Temp\ipykernel_22364\3433494013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main_models.rename(columns={'G_bottomup_tau_list': 'candidate_tau_list'}, inplace=True)


In [145]:
metrics = df_all.apply(lambda row: calculate_set_metrics(row['True_tau_list'], row['candidate_tau_list'], TOLERANCE), axis=1, result_type='expand')
df_all[['recall', 'precision', 'f1_score']] = metrics
performance_summary = df_all.groupby('model_name')[['recall', 'precision', 'f1_score']].mean()


In [146]:
df['jaccard_vs_sbs'] = df.apply(
    lambda row: calculate_modified_jaccard(row['G_bottomup_tau_list'], row['sbs_candidate_list'], TOLERANCE),
    axis=1
)
jaccard_summary = df.groupby('model_name')[['jaccard_vs_sbs']].mean()

In [147]:
final_summary = pd.concat([performance_summary, jaccard_summary], axis=1)

In [148]:
model_order = [
    'SBS_Baseline',
    'Model A (minobs=4)',
    'Model B (minobs=6)',
    'Model C (minobs=8)',
    'Model D (minobs=10)'
]

In [149]:
final_summary = final_summary.reindex(model_order)

In [150]:
final_summary

,recall,precision,f1_score,jaccard_vs_sbs
model_name,,,,
SBS_Baseline,1.0000,0.799714,0.881293,NaN
Model A (minobs=4),0.9575,0.397448,0.554383,0.470833
Model B (minobs=6),0.9800,0.431322,0.593026,0.496512
Model C (minobs=8),0.9200,0.444083,0.589235,0.492022
Model D (minobs=10),1.0000,0.428755,0.594158,0.489231


In [151]:
print("### 최종 모델별 성능 및 SBS 유사도 비교 (평균) ###")
print(f"사용한 Tolerance (허용 오차): {TOLERANCE}\n")
pd.set_option('display.width', 1000)
final_summary =final_summary.round(4)
final_summary

### 최종 모델별 성능 및 SBS 유사도 비교 (평균) ###
사용한 Tolerance (허용 오차): 5



,recall,precision,f1_score,jaccard_vs_sbs
model_name,,,,
SBS_Baseline,1.0000,0.7997,0.8813,NaN
Model A (minobs=4),0.9575,0.3974,0.5544,0.4708
Model B (minobs=6),0.9800,0.4313,0.5930,0.4965
Model C (minobs=8),0.9200,0.4441,0.5892,0.4920
Model D (minobs=10),1.0000,0.4288,0.5942,0.4892


In [152]:
output_filename = 'analysis_results_final.csv'
final_summary.to_csv(output_filename, index=False)